In [1]:
from math import *
import ROOT
from ROOT import gROOT 
import os

Welcome to JupyROOT 6.15/01


### Setting PATH and loading libraries for Delphes and ExRootAnalysis 

In [2]:
#path of folder to save the images
imgPath = './images/Spin1_Spin0/16-04-2020/'

# Delphes_Path="/home/daniel/Documents/Madgraph/MG5_aMC_v2_6_6/Delphes/"
Delphes_Path='/home/santiago/MG5_aMC_v2_6_5/Delphes'
ROOT.gSystem.AddDynamicPath(Delphes_Path)
ROOT.gSystem.Load("libDelphes.so");

# ER_Path="/home/daniel/Documents/Madgraph/MG5_aMC_v2_6_6/ExRootAnalysis/"
ER_Path="/home/santiago/MG5_aMC_v2_6_5/ExRootAnalysis/"
ROOT.gSystem.AddDynamicPath(ER_Path)
ROOT.gSystem.Load("libExRootAnalysis.so");

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
    print("Delphes classes imported")
except:
    pass

Delphes classes imported


Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootTreeReader.h
   requested to autoload type ExRootTreeReader
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootTreeWriter.h
   requested to autoload type ExRootTreeWriter
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootTreeBranch.h
   requested to autoload type ExRootTreeBranch
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootResult.h
   requested to autoload type ExRootResult
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootClassifier.h
   requested to autoload type ExRootClassifier
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootFilter.h
   requested to autoload type ExRootFil

### Functions

In [3]:
def DeltaPhi(phi1,phi2):
    """
    Description: Returns the difference of
                 the two angles phi1 and phi2
    """
    PHI=phi1-phi2
    if PHI >= pi:
        PHI -= 2*pi
    elif PHI < -1*pi:
        PHI += 2*pi
    return PHI

def MaxValueHist(h1, h2):
    """
    Description: Returns the maximum value from 
                 both histograms h1 and h2
    """
    return max(h1.GetBinContent(h1.GetMaximumBin()),
               h2.GetBinContent(h2.GetMaximumBin()))*1.1

def Histos(hs, hb,savePath=None):
    """
    Description: Plot the two histograms hs and hb
                 in the same canvas
    """
    canvas = ROOT.TCanvas()
    canvas.cd()
    hs.Scale(1/hs.Integral())
    hb.Scale(1/hb.Integral())
    hs.SetStats(0)
    hb.SetStats(0)
    hb.SetLineColor(2)
    hs.SetMaximum(MaxValueHist(hs, hb))
    hs.SetLineColor(1)
#     hs.Draw("text")
    hs.Draw()
    hb.Draw("h,same")
    
    legend = ROOT.TLegend(0.6,0.6,0.9,0.9)
    #legend.AddEntry(hss, "Signal")
    #legend.AddEntry(hbb, "Background")
    legend.Draw()
    canvas.Update()
#     canvas.Draw()
    if savePath != None:
        canvas.SaveAs(savePath)
    #gROOT.GetListOfCanvases().Draw()
    
def Histo(h,savePath=None):
    """
    Description: Plot a singular histogram h
    """
    canvas = ROOT.TCanvas()
    h.SetStats(0)
    canvas.cd()
    h.Draw("h")
    legend = ROOT.TLegend()
    legend.AddEntry(h, "Significance", "l")
    legend.Draw()
    canvas.Update()
#     canvas.Draw()
    if savePath != None :
        canvas.SaveAs(savePath)
    #canvas.Update()
    #gROOT.GetListOfCanvases().Draw()     
    
def InvariantMass(jet,i,j):
    """
    Description: Returns the invariant mass for a system
                 made out of jet[i] and jet[j] in the branch jet
    """
    jets = [ROOT.TLorentzVector(), ROOT.TLorentzVector()]
    jets[0].SetPtEtaPhiE(jet.At(i).PT, jet.At(i).Eta, jet.At(i).Phi, jet.At(i).Mass)
    jets[1].SetPtEtaPhiE(jet.At(j).PT, jet.At(j).Eta, jet.At(j).Phi, jet.At(j).Mass)
    return abs((jets[0]+jets[1]).M())


def Sig(hs, hb, ns=50000, nb=50000, L=1000, sigmab=72.38, sigmas=13.76):
    """
    Description: Returns a significance histogram
                 made out of the signal and background 
                 histograms hs and hb
    
    L: Integrated luminosity in pb^-(1)
    
    ns, nb: Number of signal and background events
    
    sigmas, sigmab: cross sections for the signal and
                    background events in pb
    """
    hs.Scale((L*sigmas/ns))
    hb.Scale((L*sigmab/nb))
    hz = hb.Clone("SigHisto")
    hz.SetTitle("{} Significance".format(hs.GetTitle()))
    for i in range(hz.GetNbinsX()):
        s = hs.Integral(i+1,hz.GetNbinsX())
        b = hb.Integral(i+1,hz.GetNbinsX())
        if i==0:
            print("s:",s)
            print("b: ",b)
            print("Z: ",s/sqrt(s+b))
        if s+b != 0:
            hz.SetBinContent(i+1,s/sqrt(s+b))
        else:
            hz.SetBinContent(i+1,0)
        hz.SetBinError(i+1,0)
    return hz

### Reading trees and selecting branches

In [4]:
# getting signal and background trees
# signal
backgroundPath = '/home/santiago/VBFDM_UdeA_CMS/scripts/preliminary_study/data/VBF_MDSimp_EWKExcluded/TEST1/'
signalPath = '/home/santiago/VBF_DMSimp_spin0_EWKExcluded/Events/run_18/'#TEST1/

fs = [] #Signal files
imgFold = [] #Images forlders
for f in os.listdir(signalPath):
    if os.path.isfile(signalPath + f) and '.root' in f:
        fs.append(ROOT.TFile(signalPath + f))
    
        #create folder to save images
        imgFold.append((imgPath+f).replace('.root',''))
        if not os.path.exists(imgFold[-1]):
            os.makedirs(imgFold[-1])
    
        
#Signal trees
s = [f.Get("Delphes") for f in fs]

# background
b = ROOT.TChain("Delphes")
b.Add(backgroundPath + "*.root")

# number of signal and background events 
nb = int(b.GetEntries())
ns = [int(k.GetEntries()) for k in s]

# number of bins to book histograms
nbBins = int(sqrt(nb))+1
nsBins = [int(sqrt(n))+1 for n in ns] 

# reading trees and selecting branches
treeB = ROOT.ExRootTreeReader(b)
treeS = [ROOT.ExRootTreeReader(t) for t in s]

jetB = treeB.UseBranch("Jet")
jetS = [tree.UseBranch("Jet") for tree in treeS]

metB = treeB.UseBranch("MissingET")
metS = [tree.UseBranch("MissingET") for tree in treeS]

# number of jets to use
numJets = 5

ExRootAnalysisDict dictionary payload:16:10: fatal error: 'ExRootAnalysis/ExRootTreeReader.h' file not found
#include "ExRootAnalysis/ExRootTreeReader.h"
         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Error in <TInterpreter::AutoParse>: Error parsing payload code for class ExRootTreeReader with content:

#line 1 "ExRootAnalysisDict dictionary payload"

#ifndef G__VECTOR_HAS_CLASS_ITERATOR
  #define G__VECTOR_HAS_CLASS_ITERATOR 1
#endif

#define _BACKWARD_BACKWARD_WARNING_H

/** \class ExRootAnalysisLinkDef
 *
 *  Lists classes to be included in cint dicitonary
 *
 *  \author P. Demin - UCL, Louvain-la-Neuve
 *
 */

#include "ExRootAnalysis/ExRootTreeReader.h"
#include "ExRootAnalysis/ExRootTreeWriter.h"
#include "ExRootAnalysis/ExRootTreeBranch.h"
#include "ExRootAnalysis/ExRootResult.h"
#include "ExRootAnalysis/ExRootUtilities.h"
#include "ExRootAnalysis/ExRootClassifier.h"
#include "ExRootAnalysis/ExRootFilter.h"

#include "ExRootAnalysis/ExRootProgressBar.h"
#include "ExRootAnalysis/E

### Book histograms

In [5]:
#---------------------------
#           signal
#---------------------------
hsDeltaEtaJet = [ROOT.TH1F("hsDeltaEtaJet_{0}".format(j), "Delta_Eta_Jet12_{0}".format(j), 50,0,10)
                for j in range(len(s))]

#Delta Eta for the two jets with highest invariant mass
hsDeltaEtaMaxIM = [ROOT.TH1F("hsDeltaEtaM_{0}".format(j), "Delta_Eta_M_{0}".format(j), 50,0,10) 
                   for j in range(len(s))]
hsMass = [ROOT.TH1F("hSMass_{0}".format(j), "M(j1,j2)_{0}".format(j), 65,0,4000)
         for j in range(len(s))]

# Highest invariant mass between to jets
hsMassMax = [ROOT.TH1F("hsMjj_{0}".format(j), "Max_M_{0}".format(j), 65,0,4000)
            for j in range(len(s))]

hsDeltaPhi = [ROOT.TH1F("hsDeltaPhi_{0}".format(j), "delta_Phi_{0}".format(j), 50, -1, 4)
              for j in range(len(s))]

# Delta phi for the two jets with highest invariant mass
hsDeltaPhiMax = [ROOT.TH1F("hsDeltaPhiMax_{0}".format(j), "Max_delta_phi_{0}".format(j), 50, -1, 4)
                 for j in range(len(s))]

hsDeltaPhiMetJet = [[ROOT.TH1F("hsDelPhiMet_j{0}_{1}".format(i,j),"dMetJ{0}_{1}".format(i,j), 50, -1, 4) 
                     for i in range(numJets)] 
                    for j in range(len(s))]

#---------------------------
#        background
#---------------------------
hbDeltaEtaJet = ROOT.TH1F("hbDeltaEtaJet", "Delta_Eta_Jet12", 50,0,10)
hbDeltaEtaMaxIM = ROOT.TH1F("hbDeltaEtaMaxIM", "Delta_Eta_MaxIM", 50,0,10)
hbMass = ROOT.TH1F("hbMass", "M(j1,j2)", 65,0,4000)
hbMassMax = ROOT.TH1F("hbMjj", "Max_M", 65,0,4000)
hbDeltaPhi = ROOT.TH1F("hbDeltaPhi", "delta_Phi", 50, -1, 4)
hbDeltaPhiMax = ROOT.TH1F("hbDeltaPhiMax", "Max_delta_phi", 50, -1, 4)
hbDeltaPhiMetJet = [ROOT.TH1F("hbDelPhiMet_j{0}".format(i),"dMetJ{0}".format(i), 50, -1, 4) 
                    for i in range(numJets)]

### Selecting events and filling histograms

In [6]:
# Loop over all signal events
for k in range(len(ns)):
    print(k)
    for event in range(ns[k]):
        #Load selected branches with data from specified event
        treeS[k].ReadEntry(event)

        # cuts
    #     # select events with MET > 200 GeV
    #     if metS.At(0).MET < 200: continue
        # select events with at least 2 jets
        if jetS[k].GetEntries() < 2: continue
        
    #     # select events with jets in opposite hemispheres 
    #     if (jetS.At(0).Eta * jetS.At(1).Eta) >= 0: continue
    #     # select events with jet1 and jet2 PT > 60
    #     if jetS.At(0).PT < 30: continue
    #     if jetS.At(1).PT < 30: continue
    #    if InvariantMass(jetS[k],0,1) < 500: continue
    #     if abs(jetS.At(0).Eta-jetS.At(1).Eta) <= 4.2: continue
    #     if abs(jetS.At(0).Eta) >= 5: continue
    #     if abs(jetS.At(1).Eta) >= 5: continue
    #     if Hs.At(0).HT <= 200 : continue
        i = 0
        j = 0
        JsMass = 0
        NJets = jetS[k].GetEntries() if jetS[k].GetEntries() <= numJets else numJets
        for n in range(NJets):
            # selecting the two jet system 
            # with the largest invariant mass
            hsDeltaPhiMetJet[k][n].Fill(abs(DeltaPhi(metS[k].At(0).Phi,jetS[k].At(n).Phi)))
            for m in range(NJets):
                if n==m:continue
                a = InvariantMass(jetS[k],n,m) 
                if a > JsMass:
                    i,j = m,n
                    JsMass = a 
        hsDeltaEtaJet[k].Fill(abs(jetS[k].At(0).Eta-jetS[k].At(1).Eta))
        hsDeltaEtaMaxIM[k].Fill(abs(jetS[k].At(i).Eta-jetS[k].At(j).Eta))
        hsMass[k].Fill(InvariantMass(jetS[k],0,1))
        hsMassMax[k].Fill(JsMass)
        hsDeltaPhi[k].Fill(abs(DeltaPhi(jetS[k].At(0).Phi,jetS[k].At(1).Phi)))
        hsDeltaPhiMax[k].Fill(abs(DeltaPhi(jetS[k].At(i).Phi,jetS[k].At(j).Phi)))

0


In file included from ClassesDict dictionary payload:34:
/home/santiago/MG5_aMC_v2_6_5/Delphes/classes/DelphesModule.h:30:10: fatal error: 'ExRootAnalysis/ExRootTask.h' file not found
#include "ExRootAnalysis/ExRootTask.h"
         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Error in <TInterpreter::AutoParse>: Error parsing payload code for class MissingET with content:

#line 1 "ClassesDict dictionary payload"

#ifndef G__VECTOR_HAS_CLASS_ITERATOR
  #define G__VECTOR_HAS_CLASS_ITERATOR 1
#endif

#define _BACKWARD_BACKWARD_WARNING_H
/*
 *  Delphes: a framework for fast simulation of a generic collider experiment
 *  Copyright (C) 2012-2014  Universite catholique de Louvain (UCL), Belgium
 *
 *  This program is free software: you can redistribute it and/or modify
 *  it under the terms of the GNU General Public License as published by
 *  the Free Software Foundation, either version 3 of the License, or
 *  (at your option) any later version.
 *
 *  This program is distributed in the hope that it wil

In [7]:
# Loop over all background events 
for event in range(nb):
    # load selected branches with data from specified event
    treeB.ReadEntry(event)
    
    # cuts
#     # select events with MET > 200 GeV
#     if metB.At(0).MET < 200: continue
    # select events with at least 2 jets
    if jetB.GetEntries() < 2: continue
    # select events with jets in opposite hemispheres 
#     if (jetB.At(0).Eta * jetB.At(1).Eta) >= 0: continue
#     # select events with jet1 and jet2 PT > 60
#     if jetB.At(0).PT <= 30: continue
#     if jetB.At(1).PT <= 30: continue
#    if InvariantMass(jetB,0,1) <= 500: continue
#     if abs(jetB.At(0).Eta-jetB.At(1).Eta) <= 4.2: continue
#     if abs(jetB.At(0).Eta) >= 5: continue
#     if abs(jetB.At(1).Eta) >= 5: continue
#     if Hb.At(0).HT <= 200 : continue
    i = 0
    j = 0
    JbMass = 0
    NJets = jetB.GetEntries() if jetB.GetEntries() <= numJets else numJets
    for n in range(NJets):
        # selecting the two jet system 
        # with the largest invariant mass
        hbDeltaPhiMetJet[n].Fill(abs(DeltaPhi(metB.At(0).Phi,jetB.At(n).Phi)))
        for m in range(NJets):
            if n==m:continue
            a = InvariantMass(jetB,n,m) 
            if a > JbMass:
                i,j = m,n
                JbMass = a 
    hbDeltaEtaJet.Fill(abs(jetB.At(0).Eta-jetB.At(1).Eta))
    hbDeltaEtaMaxIM.Fill((abs(jetB.At(i).Eta-jetB.At(j).Eta)))
    hbMass.Fill(InvariantMass(jetB,0,1))
    hbMassMax.Fill(JbMass)
    hbDeltaPhi.Fill(abs(DeltaPhi(jetB.At(0).Phi,jetB.At(1).Phi)))
    hbDeltaPhiMax.Fill(abs(DeltaPhi(jetB.At(i).Phi,jetB.At(j).Phi)))

## Delta_Eta

In [8]:
fList = open(imgPath + "fileList.txt","w+")
k=0
for f in os.listdir(signalPath):
    if os.path.isfile(signalPath + f) and '.root' in f:
        fList.write('%d\t%s\r\n'%(k,f))
        k+=1
fList.close()

for k in range(len(hsDeltaEtaJet)):
    HzsDEta = hsDeltaEtaJet[k].Clone()
    HzbDEta = hbDeltaEtaJet.Clone()
    Histos(HzsDEta,HzbDEta, imgFold[k] + '/deltaEta_j12.png')

Info in <TCanvas::Print>: png file ./images/Spin1_Spin0/16-04-2020/DMSimpSpin0_MY5000_MX1000_07042020/deltaEta_j12.png has been created


### Delta Eta maximum invariant mass jet

In [9]:
for k in range(len(hsDeltaEtaJet)):
    HzsMass = hsDeltaEtaMaxIM[k].Clone()
    HzbMass = hbDeltaEtaMaxIM.Clone()
    Histos(HzsMass,HzbMass, imgFold[k] + '/deltaEta_MaxIM.png')

Info in <TCanvas::Print>: png file ./images/Spin1_Spin0/16-04-2020/DMSimpSpin0_MY5000_MX1000_07042020/deltaEta_MaxIM.png has been created


## Invariant Mass

In [10]:
for k in range(len(hsDeltaEtaJet)):
    HsMass = hsMass[k].Clone()
    HbMass = hbMass.Clone()
    Histos(HsMass,HbMass, imgFold[k] + '/m_j1_j2.png')

Info in <TCanvas::Print>: png file ./images/Spin1_Spin0/16-04-2020/DMSimpSpin0_MY5000_MX1000_07042020/m_j1_j2.png has been created


### Maximum invariant mass

In [11]:
for k in range(len(hsDeltaEtaJet)):
    HsMass = hsMassMax[k].Clone()
    HbMass = hbMassMax.Clone()
    Histos(HsMass,HbMass, imgFold[k] + '/m_max.png')

Info in <TCanvas::Print>: png file ./images/Spin1_Spin0/16-04-2020/DMSimpSpin0_MY5000_MX1000_07042020/m_max.png has been created


### Delta Phi

In [12]:
for k in range(len(hsDeltaEtaJet)):
    HzsPhi = hsDeltaPhi[k].Clone()
    HzbPhi = hbDeltaPhi.Clone()
    Histos(HzsPhi,HzbPhi, imgFold[k] + '/deltaPhi(j1,j2).png')

Info in <TCanvas::Print>: png file ./images/Spin1_Spin0/16-04-2020/DMSimpSpin0_MY5000_MX1000_07042020/deltaPhi(j1,j2).png has been created


### Delta Phi maximum invariant mass for two jets

In [13]:
for k in range(len(hsDeltaEtaJet)):
    HzsPhi = hsDeltaPhiMax[k].Clone()
    HzbPhi = hbDeltaPhiMax.Clone()
    Histos(HzsPhi,HzbPhi, imgFold[k] + '/deltaPhi__max.png')

Info in <TCanvas::Print>: png file ./images/Spin1_Spin0/16-04-2020/DMSimpSpin0_MY5000_MX1000_07042020/deltaPhi__max.png has been created


### DeltaPhi(Met,Jet)

In [14]:
for k in range(len(hsDeltaEtaJet)):
    for n in range(numJets):
        HzsPhi = hsDeltaPhiMetJet[k][n].Clone()
        HzbPhi = hbDeltaPhiMetJet[n].Clone()
        Histos(HzsPhi,HzbPhi, imgFold[k] + '/deltaPhiMet_J{0}.png'.format(n))

Info in <TCanvas::Print>: png file ./images/Spin1_Spin0/16-04-2020/DMSimpSpin0_MY5000_MX1000_07042020/deltaPhiMet_J0.png has been created
Info in <TCanvas::Print>: png file ./images/Spin1_Spin0/16-04-2020/DMSimpSpin0_MY5000_MX1000_07042020/deltaPhiMet_J1.png has been created
Info in <TCanvas::Print>: png file ./images/Spin1_Spin0/16-04-2020/DMSimpSpin0_MY5000_MX1000_07042020/deltaPhiMet_J2.png has been created
Info in <TCanvas::Print>: png file ./images/Spin1_Spin0/16-04-2020/DMSimpSpin0_MY5000_MX1000_07042020/deltaPhiMet_J3.png has been created
Info in <TCanvas::Print>: png file ./images/Spin1_Spin0/16-04-2020/DMSimpSpin0_MY5000_MX1000_07042020/deltaPhiMet_J4.png has been created
